<a href="https://colab.research.google.com/github/atstuyuki/yolov8_osteonecrosis/blob/main/yolov8_predict_ON_20240312.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount your Google Drive Folder


In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd ./drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


# Install libraries

In [2]:
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 10.6 MB/s eta 0:00:00


# Upload your pretrained .pt file
# 訓練済みのbest.ptファイルをuploadする
## best.ptはwightsフォルダに入ってます
## using yolov8x model will takes a while
## otherwise put your best.pt model in Google Drive Folder and copy the path using right click

In [3]:
# prompt: GUIでbest.ptファイルをuploadしてpt_modelという変数に格納

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  pt_model = fn


Saving best.pt to best (1).pt
User uploaded file "best (1).pt" with length 6257433 bytes


# Upload your Image data
## repeat this cell and below when analyzing several images

In [44]:
# prompt: GUIで画像データをuploadしてimageという変数に格納

from google.colab import files

# Upload images using the file upload widget
uploaded = files.upload()

# Store the uploaded images in a variable called "image"
image = list(uploaded.keys())

# Print the names of the uploaded images
print(image)


Saving Crop前2.jpg to Crop前2.jpg
['Crop前2.jpg']


crop 6 images (size:640x640) from original image

In [51]:
# prompt: imageの縦と横のサイズを取得

from PIL import Image

# Read the image using Pillow
image_path = image[0]
img = Image.open(image_path)

# Get the image size
width, height = img.size

# Print the image size
print(f"Original Image size: {width}x{height}")
# prompt: imgからランダムに640x640の領域を6枚切り取り、imagesという変数に保存

import random
images = []
for i in range(6):
  x = random.randint(0, width - 640)
  y = random.randint(0, height - 640)
  img_crop = img.crop((x, y, x + 640, y + 640))
  images.append(img_crop)
#imagesのshapeをprint
print(f"Cropped Image size: {images[0].size}")
#imagesの要素の数をprint
print(f"Cropped Image count: {len(images)}")

Original Image size: 1920x1440
Cropped Image size: (640, 640)
Cropped Image count: 6


# Predict your Image with YOLOv8
##'runs'folder will be created in your GoogleDrive Folder
## the result will be saved in 'runs/detect/predict' folder

In [46]:
from ultralytics import YOLO
#used the uploded model
model = YOLO(pt_model)
#otherwise copy the path of your model and paste between ""
#model = YOLO("/content/drive/MyDrive/osteonecrosis_shinohara_3classes/yolov8x_20230312/weights/best.pt")
predicted = model.predict(images,show=True, save=True,save_txt=True, save_conf=True, imgsz=640, conf=0.4, show_labels=False,show_conf=False)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


0: 640x640 14 normals, 38 enmpty_lacnaes, 5.3ms
1: 640x640 8 normals, 45 enmpty_lacnaes, 5.3ms
2: 640x640 12 normals, 44 enmpty_lacnaes, 5.3ms
3: 640x640 15 normals, 18 enmpty_lacnaes, 5.3ms
4: 640x640 13 normals, 42 enmpty_lacnaes, 5.3ms
5: 640x640 12 normals, 23 enmpty_lacnaes, 5.3ms
Speed: 3.3ms preprocess, 5.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict9
6 labels saved to runs/detect/predict9/labels


# count each classes


In [47]:
# count each labels
# prompt: predicted[0]からpredicted[5]の範囲で　predicted[].boxes.cls.tolist()を取得してboxes_cls_listに追加する

boxes_cls_list = []
for i in range(6):
  boxes_cls_list.append(predicted[i].boxes.cls.tolist())
# prompt: boxes_cls_listの内容をすべて取り出して一つのリストにする
flat_boxes_cls_list = [item for sublist in boxes_cls_list for item in sublist]
print(f"Total count: {len(flat_boxes_cls_list)}")
flat_boxes_cls_list = [int(x) for x in flat_boxes_cls_list]
flat_boxes_cls_list.sort()
# prompt: boxes_cls_listの各要素を数えてprintする、さらにboxes_cls_listの長さで割り100をかけて%表記でprintする
count_dict = {}
print(model.names)
for cls in flat_boxes_cls_list:
  if cls not in count_dict:
    count_dict[cls] = 0
  count_dict[cls] += 1

for cls, count in count_dict.items():
  print(f"Class {cls}: {count}")
  print(f"Percentage: {(count / len(flat_boxes_cls_list)) * 100:.2f}%")

Total count: 284
{0: 'normal', 1: 'enmpty_lacnae'}
Class 0: 74
Percentage: 26.06%
Class 1: 210
Percentage: 73.94%


# Save count data as CSV in Google Drive Folder

In [48]:
# prompt: 各クラスの数とPersentageを表形式のデータにして保存
# CSVの名前にはimageの名前を反映

import csv

# Create a CSV file with the image name as the filename
csv_file_name = f"{image[0]}.csv"

# Write the header row
with open(csv_file_name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Class', 'Count', 'Percentage'])

    # Write the data rows
    for cls, count in count_dict.items():
        percentage = (count / len(flat_boxes_cls_list)) * 100
        writer.writerow([model.names[cls], count, f'{percentage:.2f}%'])

print(f'Saved class count data in GoogleDrive Folder as: {csv_file_name}')


Saved class count data in GoogleDrive Folder as: Crop前2.jpg.csv
